This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS, and direct air capture (DAC). Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. On the other hand, DAC leads to a true removal of CO2 from the atmosphere and not just a reduction of emissions. 

In [56]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [57]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [58]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [59]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2_Captured => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2_Atmosphere => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [60]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel


In [61]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [62]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [63]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [64]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [65]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [66]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

ng_source = Resource{NaturalGas}(;
id = :NG_Source,
node = ng_node,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
can_expand = false, 
can_retire = false,
price = ng_fuel_price,
)

Resource{NaturalGas}(Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :NG_Source, Float64[], 1.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We also define a CO2 node with both a sink and source:

In [67]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_sink = Sink{CO2}(;
node = co2_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

co2_source = Resource{CO2}(;
id = :CO2_Source,
node = co2_node,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
can_expand = false, 
can_retire = false,
price = zeros(length(time_interval(CO2))),
)

Resource{CO2}(Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_Source, Float64[], 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We define a captured CO2 node and sink to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [68]:
co2_captured_node = Node{CO2}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_captured_sink = Sink{CO2}(;
node = co2_captured_node,
id = Symbol("CO2_captured_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

Sink{CO2}(Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_captured_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [69]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 2,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_heatrate,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_fuel_CO2]
))

push!(ngcc.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [70]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 3,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc_ccs.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_ccs_heatrate,0.0,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 66400.0, 12287.0, 3.65, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [71]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                number_of_stoichiometry_balances = 1,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electrolyzer.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [1.0],
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electrolyzer.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [electrolyzer_efficiency]
))

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [72]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_heatrate,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_fuel_CO2]
))

push!(smr.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 31003.090309030904, 615.0615061506151, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, An

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [73]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 3,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr_ccs.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_ccs_heatrate,0.0,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 34003.390339033904, 645.0645064506451, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), 

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [74]:
electric_dac = Transformation{DACElectric}(;
                id = :Electric_DAC,
                time_interval = time_interval(CO2_Captured),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electric_dac.TEdges,TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [1.0,electric_dac_power_input],
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electric_dac.TEdges,TEdge{CO2}(;
    id = :CO2_input,
    node = co2_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [1.0,0.0]
))

push!(electric_dac.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{CO2}(:CO2_Captured, Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{DACElectric}(:Electric_DAC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 20.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 4.38], 0.0, Inf, 0.0, 9390.0, 7470.0, 22.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{CO2}(:CO2_input, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

Now that we have implemented different technologies, we can create the JuMP model.

In [75]:
components = [solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs; electric_dac;ng_source;co2_source;co2_sink;co2_captured_sink];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;components];

In [76]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [77]:
add_planning_variables!.(components,model);

In [78]:
add_operation_variables!.(system, model);

In [79]:
add_all_model_constraints!.(system, model);

In [80]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [81]:
println(ngcc.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] == 0


In [82]:
println(ngcc.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [83]:
println(ngcc_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] == 0


In [84]:
println(ngcc_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [85]:
println(ngcc_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2_Captured[5] == 0


In [86]:
println(electrolyzer.constraints[1].constraint_ref[1,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] == 0


In [87]:
println(smr.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] == 0


In [88]:
println(smr.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [89]:
println(smr_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] == 0


In [90]:
println(smr_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [91]:
println(smr_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2_Captured[5] == 0


In [92]:
println(electric_dac.constraints[1].constraint_ref[1,5])

vFLOW_CO2_CO2_Captured[5] - vFLOW_CO2_CO2_input[5] == 0


In [93]:
println(electric_dac.constraints[1].constraint_ref[2,5])

4.38 vFLOW_CO2_CO2_Captured[5] - vFLOW_Electricity_E[5] == 0


In [94]:
using HiGHS

In [95]:
Macro.set_optimizer(model,HiGHS.Optimizer)

In [96]:
Macro.optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
109351 rows, 100600 cols, 328053 nonzeros
100591 rows, 83080 cols, 266733 nonzeros
Presolve : Reductions: rows 100591(-188502); columns 83080(-197274); elements 266733(-464613)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 17520(3.53286e+07) 0s
      57163     3.5072693726e+09 Pr: 0(0); Du: 0(7.60836e-13) 3s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 57163
Objective value     :  3.5072693726e+09
HiGHS run time      :          3.34


The installed electrolyzer capacity is:

In [97]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The installed SMR capacity is

In [98]:
Macro.value(Macro.capacity(smr.TEdges[1]))

1039.5627

The installed SMR CCS capacity is

In [99]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

0.0

The installed solar capacity in MW is:

In [100]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [101]:
Macro.value(Macro.capacity(battery))

100.0

The installed NGCC capacity in MW is:

In [102]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

16617.0

The installed NGCC CCS capacity in MW is:

In [103]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The installed Electric DAC capacity in Tons CO2/h is:

In [104]:
Macro.value(Macro.capacity(electric_dac.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [105]:
base_emissions  = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

3.429744069856643e7

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [106]:
atm_CO2_captured  = Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

0.0

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [107]:
net_emissions = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2))) - Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

3.429744069856643e7

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [108]:
base_CO2_captured  = Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

0.0

However, if we add a net-zero CO2 cap accounting for both positive emissions from Natural gas power and H2, and negative emissions from DAC:

In [123]:
Macro.@constraint(model,CO2Cap,sum(Macro.withdrawal(co2_sink)[t] - Macro.injection(co2_source)[t] for t in time_interval(CO2)) <= 0);

In [124]:
Macro.optimize!(model)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.5074025729e+09 Pr: 1(6.85949e+07); Du: 0(1.26446e-08) 0s
        998     3.5074025764e+09 Pr: 742(3.41193e+08); Du: 0(9.2789e-06) 6s
       1846     3.5074025781e+09 Pr: 1405(3.40094e+08); Du: 0(8.12843e-05) 11s
       2724     3.5074025781e+09 Pr: 2089(3.38953e+08); Du: 0(5.40095e-05) 16s
       4109     3.5074025781e+09 Pr: 3314(3.36815e+08); Du: 0(9.7816e-06) 24s
       5040     3.5170341522e+09 Pr: 2139(3.46696e+08); Du: 0(4.84451e-08) 29s
       6243     3.5170381412e+09 Pr: 2184(3.44974e+08); Du: 0(1.62557e-07) 36s
       7449     3.5170402196e+09 Pr: 2241(3.43264e+08); Du: 0(6.12866e-08) 43s
       8657     3.5170417941e+09 Pr: 2335(2.93059e+08); Du: 0(4.84451e-08) 49s
       9868     3.5170433368e+09 Pr: 2492(2.90369e+08); Du: 0(1.91506e-07) 57s
      11081     3.5170450893e+09 Pr: 2663(2.85723e+08); Du: 0(2.70657e-07) 64s


Now, the electrolyzer capacity is:

In [125]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The SMR capacity is:

In [126]:
Macro.value(Macro.capacity(smr.TEdges[1]))

26.330700000000096

The SMR CCS capacity is:

In [127]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

1013.2319999999999

Now the solar capacity is:

In [128]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity is:

In [129]:
Macro.value(Macro.capacity(battery))

138.0

The installed NG capacity is:

In [130]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

878.0

The installed NG CCS capacity is:

In [131]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

15701.0

The installed Electric DAC capacity is:

In [132]:
Macro.value(Macro.capacity(electric_dac.TEdges[1]))

1323.4018264840186

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [133]:
Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

2.126544532807126e6

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [134]:
atm_CO2_captured  = Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

2.126544532807042e6

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [135]:
net_emissions = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2))) - Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

8.42846930027008e-8

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [137]:
Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

4.146761719728087e7

We see that in a net-zero emission case with negative emissions from DAC, a combination of Natural Gas Power CCS and SMR H2 CCS with DAC is deployed.